In [1]:
from selenium import webdriver

from selenium.webdriver.common.by import By

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome()
driver.get("https://dhcappl.nic.in/FreeText/launchbrowsejud.do#")
driver.maximize_window()

In [3]:
data = []
final_data = []
years = ['2007','2008', '2009', '2010','2011']
def court_data(driver, year):
    
    # Locate and click the link for the specific year
    page_year = driver.find_element(By.XPATH, f"""//a[@onclick="setiframe('{year}.html');"]""")
    page_year.click()
    
    # switch to the iframe which contains the data
    iframe = driver.find_element(By.XPATH, '//iframe[@id="judiframeid"]')
    driver.switch_to.frame(iframe)
    
    
    # Extract row from table
    rows = driver.find_elements(By.XPATH,'//tbody[@id="data"]/tr')
    
    # printing year after each iteration
    print("Scraping for year:", year) 
    
    #Iterate through each row in rows and extract data
    for row in rows:
        
        td_elements = row.find_elements(By.XPATH, './/td')
        
        row_data = [td.text for td in td_elements]
        
        # Extracting the link which is in the 4 element.
        link = td_elements[4].find_element(By.XPATH, './/a').get_attribute('href')
        row_data.append(link)
        
        data.append(row_data)
        
    #Switch back the original content    
    driver.switch_to.default_content()
    
    return data

for year in years:
    final_data.extend(court_data(driver, year))

Scraping for year: 2007
Scraping for year: 2008
Scraping for year: 2009
Scraping for year: 2010
Scraping for year: 2011


In [4]:
driver.quit()

In [5]:
df_list = []
for list_ in data:
    new_row = {'CASE NO': list_[0], 'CATEGORY': list_[1], 'PARTY NAME': list_[2], 'JUDGEMENT DATE': list_[3], 'JUDGEMENT PDF LINK':list_[5]}
    df_list.append(new_row)
    
df = pd.DataFrame(df_list)
df.head()

,CASE NO,CATEGORY,PARTY NAME,JUDGEMENT DATE,JUDGEMENT PDF LINK
0,W.P.(C) 5108/1993,INCOME TAX MATTERS,M/s Puran Mal & Sons and another VS Union of I...,24-10-07,https://dhccaseinfo.nic.in/jupload/dhc/MBL/jud...
1,FAO(OS) 276/1995,NOT ENTERED,M/S. CAPRICORN ONE & ANOTHER VS RAGHBIR SINGH ...,15-05-07,https://dhccaseinfo.nic.in/jupload/dhc/SKN/jud...
2,W.P.(C) 2165/1995,P.I.L. MATTERS,P.CHAKRABORTY VS UOI & ORS.,14-03-07,https://dhccaseinfo.nic.in/jupload/dhc/620/jud...
3,CRL.A. 38/1991,CRIMINAL APPEALS,State VS Kaptan Singh,06-11-07,https://dhccaseinfo.nic.in/jupload/dhc/PNJ/jud...
4,FAO 58/1994,MOTOR ACCIDENT CLAIM MATTERS,U.P.Roadways VS Smt. Vijay Devi & Ors.,21-05-07,https://dhccaseinfo.nic.in/jupload/dhc/PNJ/jud...


In [7]:
df.sample()

,CASE NO,CATEGORY,PARTY NAME,JUDGEMENT DATE,JUDGEMENT PDF LINK
9518,ITA 1232/2008,Assessment/Re-assessment/Block assessment,THE COMMISSIONER OF INCOME TAX-III VS SPORTKIN...,19-08-09,https://dhccaseinfo.nic.in/jupload/dhc/VJM/jud...


In [8]:
data = df.to_csv("C:/Users/ADMIN/Desktop/New folder/Project/Court scrape/Court_data_2007-2011.csv")

In [9]:
df1 = pd.read_csv("C:/Users/ADMIN/Desktop/New folder/Project/Court scrape/Court_data_2007-2011.csv")
df1.sample()

,Unnamed: 0,CASE NO,CATEGORY,PARTY NAME,JUDGEMENT DATE,JUDGEMENT PDF LINK
2391,2391,FAO(OS) 412/2006,SPECIFIC PERFORMANCE OF CONTRACT MATTERS,ROYAL NEPAL AIRLINES CORPORATION VS ARUN JAIN,06-02-08,https://dhccaseinfo.nic.in/jupload/dhc/SKN/jud...


In [10]:
df = df1.copy()

In [11]:
df.rename(columns={'Unnamed: 0': 'S.NO'}, inplace=True)

In [12]:
df.sample()

,S.NO,CASE NO,CATEGORY,PARTY NAME,JUDGEMENT DATE,JUDGEMENT PDF LINK
9633,9633,W.P.(C) 8365/2008,DIVISION BENCH CASE NOT FALLING IN ANY SPECIFI...,JAY THAREJA & ANR. VS LIEUTENANT GOVERNOR & ANR.,15-04-09,https://dhccaseinfo.nic.in/jupload/dhc/SKK/jud...


In [13]:
#Spliting the Party name column into party name and respondent's 
df['PARTY NAME'], df['RESPONDENT"S'] = df['PARTY NAME'].str.split(" VS ", 1).str 

In [14]:
# Reording the columns name
df = df[['S.NO', 'CASE NO', 'CATEGORY', 'PARTY NAME','RESPONDENT"S','JUDGEMENT DATE', 'JUDGEMENT PDF LINK']]

In [15]:
df.sample()

,S.NO,CASE NO,CATEGORY,PARTY NAME,"RESPONDENT""S",JUDGEMENT DATE,JUDGEMENT PDF LINK
10458,10458,FAO 365/1999,MOTOR ACCIDENT CLAIM MATTERS,SAT PRAKASH & ORS,JAGDISH & ORS,26-03-10,https://dhccaseinfo.nic.in/jupload/dhc/JRM/jud...


In [16]:
df.to_csv("C:/Users/ADMIN/Desktop/New folder/Project/Court scrape/Court_data_final_2007-2011.csv", encoding='utf-8', index=False)